In [1]:
import os
import sys
import random
import pickle

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from tqdm import tqdm
from pandarallel import pandarallel

import pymorphy2
import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize

from sklearn.model_selection import train_test_split

SEED = 1
random.seed(SEED)

pd.set_option('display.max_colwidth', 255)
tqdm.pandas()
pandarallel.initialize(progress_bar=True, nb_workers=8, use_memory_fs=False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
abbr = pd.read_csv("../data/abbr.csv")
lenta = pd.read_csv("../data/lenta.csv")

<ipython-input-2-e66670f0ab96>:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  lenta = pd.read_csv("../data/lenta.csv")


In [4]:
class AbbrInfo:
    def __init__(self, abbr_id, abbr, abbr_count):
        self.abbr_id = abbr_id 
        self.abbr = abbr
        self.abbr_count = abbr_count

ABBR_LIST_KEY = "<ABBR_LIST_KEY>"
        
def create_abbr_tree(abbr, abbr_list_key = ABBR_LIST_KEY):        
    tree = {}
    for norm_desc, norm_abbr, abbr_id, abbr_count in abbr[["desc_norm", 
                                                           "abbr_norm", 
                                                           "abbr_id", 
                                                           "abbr_count"]].values:
        words = norm_desc.split(" ")

        curr_tree = tree
        for word in words:
            if word not in curr_tree:
                curr_tree[word] = {}
            curr_tree = curr_tree[word]


        if abbr_list_key not in curr_tree:
            curr_tree[abbr_list_key] = []

        curr_tree[abbr_list_key].append(AbbrInfo(abbr_id, norm_abbr, abbr_count))
    return tree

In [5]:
def choice_abbr(abbr_list: list, 
                weighted_choice: bool = True, 
                add_to_zeros: float = 0):
    abbr_counts = []
    
    if weighted_choice:
        for abbr_info in abbr_list:
            cnt = abbr_info.abbr_count
            if cnt == 0:
                cnt = add_to_zeros
            abbr_counts.append(cnt)
    else:
        abbr_counts = None
    
    
    return random.choices(abbr_list, weights=abbr_counts, k=1)[0]

In [ ]:
OUTSIDE_LABEL = "_"
BEGIN_LABEL = "B"
END_LABEL = "E"
INSIDE_LABEL = "I"
ONE_WORD_LABEL = "W"

def get_text_labels(text, 
                    abbr_tree, 
                    weighted_choice: bool = None, 
                    add_to_zeros: float = None):
    text = text.split(" ")
    labels = [OUTSIDE_LABEL for i in range(len(text))]

    curr_node = abbr_tree
    desc_start = None

    word_i = 0
    while word_i < len(text):
        curr_i = word_i
        while curr_i < len(text) and text[curr_i] in curr_node:
            curr_node = curr_node[text[curr_i]]
            curr_i += 1

        if ABBR_LIST_KEY in curr_node: 

            abbr_id = choice_abbr(curr_node[ABBR_LIST_KEY], weighted_choice, add_to_zeros).abbr_id

            labels[word_i] = f"{BEGIN_LABEL}-{abbr_id}"
            for j in range(word_i + 1, curr_i - 1): 
                labels[j] = f"{INSIDE_LABEL}-{abbr_id}"
            labels[curr_i - 1] = f"{END_LABEL}-{abbr_id}"

            if word_i == curr_i - 1:
                labels[word_i] = f"{ONE_WORD_LABEL}-{abbr_id}"

            word_i = curr_i - 1

        curr_node = abbr_tree
        word_i += 1
    return " ".join(labels)

get_text_labels(lenta.text_norm.iloc[0], abbr_tree)

In [8]:
text = lenta.text.iloc[0]
morph = pymorphy2.MorphAnalyzer(lang="ru", 
                                units=[pymorphy2.units.DictionaryAnalyzer()])
tokenized_text = []
tokenized_norm = []
for word in word_tokenize(text):
    parse_list = morph.parse(str(word))
    if parse_list != []:
        norm_form = parse_list[0].normal_form
    else:
        norm_form = word
    tokenized_text.append(word)
    tokenized_norm.append(norm_form)

In [10]:
" ".join(tokenized_text)

'Бои у Сопоцкина и Друскеник закончились отступлением германцев . Неприятель , приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью . В артиллерийском бою принимают участие тяжелые калибры . С раннего утра 14 сентября огонь достиг значительного напряжения . Попытка германской пехоты пробиться ближе к крепости отражена . В Галиции мы заняли Дембицу . Большая колонна , отступавшая по шоссе от Перемышля к Саноку , обстреливалась с высот нашей батареей и бежала , бросив парки , обоз и автомобили . Вылазки гарнизона Перемышля остаются безуспешными . При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей , захватываются новые партии пленных , орудия и прочая материальная часть . На перевале Ужок мы разбили неприятельский отряд , взяли его артиллерию и много пленных и , продолжая преследовать , вступили в пределы Венгрии . « Русский инвалид » , 16 сентября 1914 года .'

In [11]:
" ".join(tokenized_norm)

'бой у Сопоцкина и Друскеник закончиться отступление германец . неприятель , приблизиться с север к Осовцу начать артиллерийский борьба с крепость . в артиллерийский бой принимать участие тяжёлый калибр . с ранний утро 14 сентябрь огонь достигнуть значительный напряжение . попытка германский пехота пробиться близкий к крепость отразить . в Галиции мы занять Дембицу . больший колонна , отступать по шоссе от перемышль к Саноку , обстреливаться с высота наш батарея и бежать , бросить парка , обоз и автомобиль . вылазка гарнизон перемышль оставаться безуспешный . при продолжаться отступление австриец обнаруживаться полный перемешивание они часть , захватываться новый партия пленный , орудие и прочий материальный часть . на перевал Ужок мы разбить неприятельский отряд , взять он артиллерия и много пленный и , продолжать преследовать , вступить в предел венгрия . « русский инвалид » , 16 сентябрь 1914 год .'

In [12]:
assert len(tokenized_norm) == len(tokenized_text)

Бои бой
у у
Сопоцкина Сопоцкина
и и
Друскеник Друскеник
закончились закончиться
отступлением отступление
германцев германец
. .
Неприятель неприятель
, ,
приблизившись приблизиться
с с
севера север
к к
Осовцу Осовцу
начал начать
артиллерийскую артиллерийский
борьбу борьба
с с
крепостью крепость
. .
В в
артиллерийском артиллерийский
бою бой
принимают принимать
участие участие
тяжелые тяжёлый
калибры калибр
. .
С с
раннего ранний
утра утро
14 14
сентября сентябрь
огонь огонь
достиг достигнуть
значительного значительный
напряжения напряжение
. .
Попытка попытка
германской германский
пехоты пехота
пробиться пробиться
ближе близкий
к к
крепости крепость
отражена отразить
. .
В в
Галиции Галиции
мы мы
заняли занять
Дембицу Дембицу
. .
Большая больший
колонна колонна
, ,
отступавшая отступать
по по
шоссе шоссе
от от
Перемышля перемышль
к к
Саноку Саноку
, ,
обстреливалась обстреливаться
с с
высот высота
нашей наш
батареей батарея
и и
бежала бежать
, ,
бросив бросить
парки парка
, ,
обоз обоз


In [14]:
OUTSIDE_LABEL = "_"
BEGIN_LABEL = "B"
END_LABEL = "E"
INSIDE_LABEL = "I"
ONE_WORD_LABEL = "W"

def get_text_labels(text, 
                    abbr_tree, 
                    weighted_choice: bool = None, 
                    add_to_zeros: float = None):
    text = text.split(" ")
    labels = [OUTSIDE_LABEL for i in range(len(text))]

    curr_node = abbr_tree
    desc_start = None

    word_i = 0
    while word_i < len(text):
        curr_i = word_i
        while curr_i < len(text) and text[curr_i] in curr_node:
            curr_node = curr_node[text[curr_i]]
            curr_i += 1

        if ABBR_LIST_KEY in curr_node: 

            abbr_id = choice_abbr(curr_node[ABBR_LIST_KEY], weighted_choice, add_to_zeros).abbr_id

            labels[word_i] = f"{BEGIN_LABEL}-{abbr_id}"
            for j in range(word_i + 1, curr_i - 1): 
                labels[j] = f"{INSIDE_LABEL}-{abbr_id}"
            labels[curr_i - 1] = f"{END_LABEL}-{abbr_id}"

            if word_i == curr_i - 1:
                labels[word_i] = f"{ONE_WORD_LABEL}-{abbr_id}"

            word_i = curr_i - 1

        curr_node = abbr_tree
        word_i += 1
    return " ".join(labels)

In [15]:
abbr_tree = create_abbr_tree(abbr)

In [21]:
labels

'_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ W-1167 _ _ _ _ _ W-567 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ W-748 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ W-974 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ W-310 _ _ _'

In [23]:
labels = get_text_labels(" ".join(tokenized_norm), abbr_tree)
for i in range(len(tokenized_text)):
    print(tokenized_text[i], tokenized_norm[i], labels.split(" ")[i])

Бои бой _
у у _
Сопоцкина Сопоцкина _
и и _
Друскеник Друскеник _
закончились закончиться _
отступлением отступление _
германцев германец _
. . _
Неприятель неприятель _
, , _
приблизившись приблизиться _
с с _
севера север _
к к _
Осовцу Осовцу _
начал начать _
артиллерийскую артиллерийский _
борьбу борьба _
с с _
крепостью крепость _
. . _
В в _
артиллерийском артиллерийский _
бою бой _
принимают принимать _
участие участие _
тяжелые тяжёлый _
калибры калибр W-1167
. . _
С с _
раннего ранний _
утра утро _
14 14 _
сентября сентябрь W-567
огонь огонь _
достиг достигнуть _
значительного значительный _
напряжения напряжение _
. . _
Попытка попытка _
германской германский _
пехоты пехота _
пробиться пробиться _
ближе близкий _
к к _
крепости крепость _
отражена отразить _
. . _
В в _
Галиции Галиции _
мы мы _
заняли занять _
Дембицу Дембицу _
. . _
Большая больший _
колонна колонна _
, , _
отступавшая отступать _
по по _
шоссе шоссе _
от от _
Перемышля перемышль _
к к _
Саноку Саноку _
, 

In [25]:
def replace_word_by_abbr(text, labels, abbr, p_replace: float = 0.2):
    text = text.split(" ")
    labels = labels.split(" ")
    
    new_text = []
    new_labels = []

    i = 0
    while i < len(text):
        label = labels[i]
        if label == OUTSIDE_LABEL:
            new_text.append(text[i])
            new_labels.append(OUTSIDE_LABEL)

        mode = label[0]

        if mode in [ONE_WORD_LABEL, BEGIN_LABEL]:
            abbr_id = int(label[2:])
            replaced = random.choices([False, True], weights=[(1 - p_replace), p_replace])[0]
            if replaced:
                norm_abbr = abbr[abbr.abbr_id == abbr_id].abbr_norm.iloc[0].split(" ")
                
                if len(norm_abbr) == 1:
                    new_text.append(norm_abbr[0])
                    new_labels.append(f"{ONE_WORD_LABEL}-{str(abbr_id)}")
                else:
                    new_text.append(norm_abbr[0])
                    new_labels.append(f"{BEGIN_LABEL}-{str(abbr_id)}")
                    for word in norm_abbr[1:-1]:
                        new_text.append(word)
                        new_labels.append(f"{INSIDE_LABEL}-{str(abbr_id)}")
                    new_text.append(norm_abbr[-1])
                    new_labels.append(f"{END_LABEL}-{str(abbr_id)}")

            while i < len(text) and labels[i] != OUTSIDE_LABEL and int(labels[i][2:]) == abbr_id:
                if not replaced:
                    new_text.append(text[i])
                    new_labels.append(OUTSIDE_LABEL)
                i += 1
        else:
            i += 1

    new_text = " ".join(new_text)
    new_labels = " ".join(new_labels)
    
    return pd.Series({"new_text": new_text, "new_labels": new_labels})

In [26]:
replaced_text = get_text_labels(" ".join(tokenized_norm), labels, abbr_tree)

TypeError: string indices must be integers

In [27]:
text = lenta.text.iloc[i]
labels = get_text_labels(text, abbr_tree)

replaced_series = replace_word_by_abbr(text, labels, abbr)

'_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ W-1167 _ _ _ _ _ W-567 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ W-748 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ W-974 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ W-310 _ _ _'